In [1]:
import requests
import sys
import gzip
import csv
import os
import pandas as pd
from tempfile import TemporaryDirectory 
from pathlib import Path, PurePath
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

В процессе подготовки к скрейпингу объявлений с сайта auto.ru первым делом был открыт файл https://auto.ru/robots.txt, где обнаружена ссылка на карту сайта https://auto.ru/sitemapindex.xml. Это xml-файл, содержащий ссылки на архивы с тектовыми файлами, в которых находятся ссылки на страницы объявлений.
Нам нужны не все из них (не рассматриваем коммерческий транспорт и подборки объявлений, потому что в подборках находятся ссылки на те же самые объявления). Для получения ссылок в виде csv файла, который можно было бы передать в scrapy можно воспользоваться этим ноутбуком.

In [2]:
output = Path('.').resolve().parents[0].joinpath('input', 'links.csv')
sitemap_link='https://auto.ru/sitemapindex.xml'

In [3]:
temp_fold = TemporaryDirectory()
temp_path = Path(temp_fold.name)
r = requests.get(sitemap_link)
text = r.content.decode()
soup = BeautifulSoup(text, 'lxml')
links_to_gz = []
i = 0
for element in soup.find_all('loc'):
    links_to_gz.append(element.text.strip())
    i += 1
print(f'Found {i} link lists as gzipped xml files.')

Found 51 link lists as gzipped xml files.


In [4]:
offers_cars = [x for x in links_to_gz if 'offers_cars' in x]

In [5]:
i=0
for link in tqdm(offers_cars):
    tempfile = temp_path / link.split('/')[-1]
    if not tempfile.is_file():
        r = requests.get(link, allow_redirects=True)
        with tempfile.open('wb') as file:
            file.write(r.content)
        i+=1
print(f'Downloaded {i} link files')

  0%|          | 0/8 [00:00<?, ?it/s]

Downloaded 8 link files


In [6]:
i = 0
for gz in tqdm(list(temp_path.iterdir())):
    gz = gz.resolve()
    if gz.suffix == '.gz':
        with gzip.open(gz) as archive:
            with open(gz.parent / gz.stem, 'wb') as output_file:
                output_file.write(archive.read())
        gz.unlink()
    i += 1
print(f'Unzipped {i} txt files.')

  0%|          | 0/8 [00:00<?, ?it/s]

Unzipped 8 txt files.


In [7]:
offers_list = []
for txt in tqdm(list(temp_path.iterdir())):
    offers_counter = 0
    with open(txt, 'r') as f:
        file_content = f.read().splitlines()
        offers_list += file_content
        print(f'Found {len(file_content)} links in file {txt}, totally {len(offers_list)} links')
# write this list to csv
output_path = Path(output).resolve()
with output_path.open('w+', newline='') as f:
    wr = csv.writer(f)
    for line in offers_list:
        wr.writerow([line])
print(f'Output written to file {str(output_path)}')
# cleanup temp folder
temp_fold.cleanup()


  0%|          | 0/8 [00:00<?, ?it/s]

Found 50000 links in file /tmp/tmp8jw7idg2/sitemap_offers_cars_5.txt, totally 50000 links
Found 50000 links in file /tmp/tmp8jw7idg2/sitemap_offers_cars_6.txt, totally 100000 links
Found 50000 links in file /tmp/tmp8jw7idg2/sitemap_offers_cars_1.txt, totally 150000 links
Found 50000 links in file /tmp/tmp8jw7idg2/sitemap_offers_cars_4.txt, totally 200000 links
Found 50000 links in file /tmp/tmp8jw7idg2/sitemap_offers_cars_2.txt, totally 250000 links
Found 50000 links in file /tmp/tmp8jw7idg2/sitemap_offers_cars_7.txt, totally 300000 links
Found 49274 links in file /tmp/tmp8jw7idg2/sitemap_offers_cars_8.txt, totally 349274 links
Found 50000 links in file /tmp/tmp8jw7idg2/sitemap_offers_cars_3.txt, totally 399274 links
Output written to file /home/rivfader/shared/GoogleDrive/Prog/Skillfactory/SF_training/module_6/input/links.csv


На основе этих ссылок был написан скрейпер, собирающий информацию со страниц объявлений из этих ссылок, и формирующий датасет. 
Скрейпер запускается из папки ./scraping/autoru_scrapy командой\
``scrapy crawl -a links_file=(ВОТ СЮДА ПОСТАВИТЬ ПУТЬ К РЕЗУЛЬТАТУ РАБОТЫ GET_LINKS.IPYNB) -o ../dataset.csv(или другой путь к выходному датасету) -L ERROR autoru_scraper``\
Cтрока с дефолтными настройками в ячейке ниже, написано для запуска из bash.

In [8]:
%%bash
cd .. &&
touch './input/dataset_with_electro.csv' &&
output_file_path=$(realpath ./input/dataset_with_electro.csv) &&
links_file=$(realpath ./input/links.csv) && 
cd ./scraping/autoru_scrapy && 
scrapy crawl -a links_file=$links_file -o $output_file_path -L ERROR autoru_scraper

Также нам потребуется собрать данные по ссылкам на объявления из теста. Это необходимо для упрощения сведения теста и трейна к одному формату. Никаких подсматриваний в цену.

In [9]:
input_path = Path('.').resolve().parents[0].joinpath('input')
test_links = pd.read_csv(input_path.joinpath('test.csv'), usecols=['car_url'])
test_links.to_csv(input_path.joinpath('test_links.csv'), header = False, index = False)

In [10]:
%%bash
cd .. &&
touch './input/test_scraped.csv' &&
output_file_path=$(realpath ./input/test_scraped.csv) &&
links_file=$(realpath ./input/test_links.csv) && 
cd ./scraping/autoru_scrapy && 
scrapy crawl -a links_file=$links_file -o $output_file_path -L ERROR autoru_scraper

2021-03-17 13:49:02 [scrapy.core.scraper] ERROR: Error processing {'body_type': None,
 'brand': 'Toyota',
 'catalog_link': None,
 'color': 'чёрный',
 'complectation': ['Ксеноновые/Биксеноновые фары',
                   'Противотуманные фары',
                   'Датчик дождя',
                   'Датчик света',
                   'Электрообогрев боковых зеркал',
                   'Диски 17',
                   'Рейлинги на крыше',
                   'Легкосплавные диски',
                   'Центральный замок',
                   'Иммобилайзер',
                   'Аудиосистема',
                   'Третий ряд сидений',
                   'Подогрев передних сидений',
                   'Регулировка сиденья водителя по высоте',
                   'Регулировка передних сидений по высоте',
                   'Электрорегулировка сиденья водителя',
                   'Электрорегулировка передних сидений',
                   'Тонированные стекла',
                   'Электростеклоподъёмники